# Developing an AI application (orig)

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
from __future__ import print_function, division
from collections import OrderedDict
import json
import matplotlib.pyplot as plt
import numpy as np

import os
from PIL import Image
import requests
import tarfile
import torch
from torch import nn, optim
import torch.nn.functional as F

from torchvision import datasets, transforms, models
from tqdm import tqdm as tqdm
%matplotlib inline

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

### File utilities
Download the training data and exand it into the data directory

In [ ]:
def download_file(data_url, dest_dir='./'):
    fn = os.path.join(data_dir, os.path.split(data_url)[-1])
    r = requests.get(data_url, stream=True)
    size = int(r.headers.get('Content-Length',0))

    with open(fn, 'wb') as f:
        with tqdm(total=size, unit = 'B', unit_scale=True) as t:
            for chunk in r.iter_content(1024):
                if chunk:
                    f.write(chunk)
                    t.update(len(chunk))
    return fn

def get_data(url, data_dir, train_dirname = '/train'):
    train_dir = data_dir + train_dirname
    if os.path.exists(train_dir):
        return
    elif not os.path.exists(data_dir):
        os.makedirs(data_dir)
    fn = download_file(url)
    with tarfile.open(fn) as tar:
        tar.extractall(path=data_dir)
    if os.path.isfile(fn): os.remove(fn)

### Dataset download and extract
Downloads the dataset (already structured into directories as required) in a tar.gz file and extracts it into the appropriate directory, ready for training, validation and testing.

In [ ]:
data_url = 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

# Download flower data
get_data(data_url, data_dir)

### Transformations and loading

* data transforms, conversion to tensor and normalization
* Use ImageFolder and transform to create datasets for training, validation and testing
* Create dataloaders from the datasets

_Note: This notebook is using the inception v3 model requires a slightly different input image size from most of the other models in the pyTorch model zoo._

In [ ]:
# Define your transforms for the training, validation, and testing sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        #transforms.RandomResizedCrop(224),  # not inception
        transforms.RandomResizedCrop(299),  # inception
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        #transforms.Resize(256),     # not inception
        #transforms.CenterCrop(224), # not inception
        transforms.Resize(331),      # inception
        transforms.CenterCrop(299),  # inception
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        #transforms.Resize(256),       # not inception
        #transforms.CenterCrop(224),  # not inception
        transforms.Resize(331),      # inception
        transforms.CenterCrop(299),  # inception
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}
# Load the datasets with ImageFolder
image_datasets = {
    'train' : datasets.ImageFolder(
        train_dir, transform=data_transforms['train']),
    'valid' : datasets.ImageFolder(
        valid_dir, transform=data_transforms['valid']),
    'test' : datasets.ImageFolder(
        test_dir, transform=data_transforms['test'])
}
# Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train' : torch.utils.data.DataLoader(
        image_datasets['train'], batch_size=64, shuffle=True),
    'valid' : torch.utils.data.DataLoader(
        image_datasets['valid'], batch_size=64, shuffle=True),
    'test' : torch.utils.data.DataLoader(
        image_datasets['test'], batch_size=64, shuffle=True)
}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

1. Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
1. Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
1. Train the classifier layers using backpropagation using the pre-trained network to get the features
1. Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

## Define new network
Create new network that refines the Inception v3 pre-trained network to extract features from the images.

In [ ]:
class NewModel(nn.Module):
    def __init__(self, n_classes):
        super(NewModel, self).__init__()
        
        ### Inception v3 based refined model
        pt_model = models.inception_v3(pretrained=True)
        pt_model.aux_logits=False
        
        # Freeze parameters
        for p in pt_model.parameters():
            p.requires_grad = False
        
        # New feed forward network as classifier
        # Trying one FC layer...do more make it better?
        in_features = pt_model.fc.in_features
        h_units = in_features
        self.classifier = nn.Sequential(OrderedDict([
            ('fc0', nn.Linear(in_features, 1024)),
            ('relu0', nn.ReLU()),
            ('drop0', nn.Dropout(.5)),
            ('fc', nn.Linear(1024, n_classes))
        ]))
        # replace the last fc layer with the classifier
        pt_model.fc = self.classifier
        self.model = pt_model
        
    def forward(self, x):
        return self.model(x)

## PyTorch Primer 
**(aka. stuff I wish I knew and/or remembered, but wasted a LOT of time on!)**

I'd never used PyTorth prior to the nanodegree, and I found myself struggling to do relatively simple operations due to the way that it handles Tensor operations (aka. type issues).   Hopefully the following simple list of functions and hints I found useful will help the next person so they can spend fewer hours and have less headaches trying to do 'simple' things (and get on with doing some transfer learning).

** HINTS ** (and easy oversights)
* Take extra care with device: it matters where your tensors are!

** Operations **
* .float, .int, .add, .sub, .mul, and .div have inplace versions with a trailing "\_" (E.g. ".add" becomes ".add_")
* .eq (NOT .equal): eq checks if elements are the same, equal checks if the whole tensor is the same
* .topk: gets the k largest and smallest 
** Manipulations **
* .view: reshape a tensor, # of elements is the unchanged (hint: use -1 for the size and it'll be inferred...if possible)
* .expand, .expand_as: creates a new view with singleton dims expanded
* .size: Gets the size of the tensor
* .transpose, .t: .t assumes dim 0 and 1 are transposed, and it's LOWER CASE

## Helpers

Some handy code for the application.

In [ ]:
def accuracy_score (output, target, top_k=(1,)):
    max_k = max(top_k)
    n_batch = target.size(0)
    
    _, preds = output.topk(max_k, 1, True, True)
    preds = preds.transpose(0,1)
    correct = preds.eq(target.view(1, -1).expand(preds.size()))
    
    retval = []
    for k in top_k:
        correct_k = correct[:k].view(-1).float().sum(0)
        retval.append(correct_k.mul_(100.0 / n_batch))
        
    return retval

In [ ]:
class ValueCache(object):
    '''Used to stored/calc the values'''
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.value = 0.0
        self.sum = 0.0
        self.count = 0
        self.mean = 0.0
    
    def update(self, value, n=1):
        self.value = value
        self.sum += value * n
        self.count += n
        self.mean = self.sum/self.count

In [ ]:
def get_postfix(losses, accuracy, action='Training'):
    return ('{act} Loss {loss.value:.4f} (avg:{loss.mean:.4f})\t{act} Accuracy: {acc.value:.4f}(avg:{acc.mean:.4f})'
            .format(loss=losses, acc=accuracy, act=action))

In [ ]:
# Save the checkpoint
def save_checkpoint(model, filename='./checkpoint.pth'):
    model_state = {
    'epoch': model.epoch,
    'state_dict': model.state_dict(),
    'optimizer_state_dict': model.optimizer_state,
    'best_acc': model.best_acc,
    'class_to_idx': model.class_to_idx,
    'n_classes': model.n_classes,
    }
    
    torch.save(model_state, filename)

## Model training and validation

In [ ]:
def train(model, train_loader, criterion, optimizer, epoch=1, device='cpu'):
    print_freq = 4
    losses = ValueCache()
    acc = ValueCache()
    # training state
    model.to(device)
    model.train()
    # wrap this in tqdm to get a progress bar and updates
    with tqdm(train_loader, unit='batch', postfix=None) as t:
        for i, (images, labels) in enumerate(t):
            images, labels = images.to(device), labels.to(device)
            # Forward
            output = model(images)
            loss = criterion(output, labels)
            
            # cache the accuracy and loss
            losses.update(loss.item(), images.size(0))
            accuracy = accuracy_score(output.data, labels)
            acc.update(accuracy[0], images.size(0))
            
            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if i % print_freq == (print_freq - 1):
                t.postfix = get_postfix(losses, acc)
                t.update()
    pass

In [ ]:
def validate(model, valid_loader, criterion, device='cpu', action='Validation'):
    print_freq = 4
    losses = ValueCache()
    acc = ValueCache()
    # eval state
    model.to(device)
    model.eval()
    with tqdm(valid_loader, unit='batch', postfix=None) as t:
        for i, (images, labels) in enumerate(t):
            images, labels = images.to(device), labels.to(device)
            # Forward
            output = model(images)
            loss = criterion(output, labels)

            # cache the accuracy and loss
            losses.update(loss.item(), images.size(0))
            acc.update(accuracy_score(output.data, labels)[0], images.size(0))
            
            if i % print_freq == (print_freq-1):
                t.postfix = get_postfix(losses, acc, action)
    return acc.mean

In [ ]:
### Main program Start
if torch.cuda.is_available():
    device = 'cuda'
else: 
    device = 'cpu'
# Create an instance of the new model
n_classes = len(datasets.folder.find_classes(train_dir)[0])
model = NewModel(n_classes=n_classes)
model.n_classes = n_classes

# loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), 
                      lr=0.1, momentum=0.8, weight_decay=0.0, nesterov=False)
# global vars
epochs = 5
best_acc = 0

for epoch in range(epochs):
    print('Epoch: {0}/{1}'.format(epoch + 1, epochs))
    # Train and validate for an epoch
    train(model , dataloaders['train'], loss_func, optimizer, epoch, device)
    accuracy = validate(model, dataloaders['valid'], loss_func, device)
    best_acc = max(accuracy, best_acc)

    # Save states to the model for checkpoints
    model.epoch = epoch + 1
    model.best_acc = best_acc
    model.optimizer_state = optimizer.state_dict()

model.class_to_idx = image_datasets['train'].class_to_idx

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# Do validation on the test set
print('\n*****\nTest set accuracy: {}'.format(validate(model, dataloaders['test'], loss_func, device, action='Test')))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Save model to checkpoint
save_checkpoint(model)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(model=None, filename='./checkpoint.pth'):
    if model is None:
        model = NewModel(n_classes=len(datasets.folder.find_classes(train_dir)[0]))
    
    model_state = torch.load(filename, map_location='cpu')
    model.load_state_dict(model_state['state_dict'])
    optimizer=optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), 
                      lr=0.1, momentum=0.8, weight_decay=0.0, nesterov=False)
    optimizer.load_state_dict(model_state['optimizer_state_dict'])

    return model, optimizer

In [ ]:
# Test the saved checkpoint and reloaded model
reloaded, reoptimizer = load_checkpoint()
print('\n*****\nReloaded test accuracy: {}'.format(validate(reloaded, dataloaders['test'], nn.CrossEntropyLoss(), device, action='Test')))

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image, out_size=299):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
        ? could have done using TorchVision, why not if already a requirement?
    '''
    # resize/thumbnail to 'out_size' + 32 pixels on shortest side
    w, h = image.size
    scale = (out_size + 32)/ np.min([w,h])
    image = image.resize((int(scale * w),int(scale * h)))

    # center crop image to out_size square
    left = int((image.size[0]/2) - (out_size/2))
    upper = int((image.size[1]/2) - (out_size/2))
    right = left + out_size
    lower = upper + out_size  # 'lower' is higher num, axis points down
    image = image.crop((left, upper, right, lower))

    # scale + normalize
    image = np.array(image)/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = (image - mean) / std

    # Move color channel from last to first position
    image = image.transpose((2,0,1))

    return image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
        
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = image * std + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    ax.imshow(image)
    ax.set_xticks([])
    ax.set_yticks([])
    if title is None: title = 'Title'
    ax.set_title(title)
    return ax

In [ ]:
# test process_image function a 'random' image  (I just picked one)
pil_image = Image.open('./flowers/test/74/image_01173.jpg')
imshow(process_image(pil_image), title=cat_to_name['74']);

**Yay!  It looks like a flower picture!**

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
# Implement the code to predict the class from an image file
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    if not os.path.isfile(image_path):
        raise FileNotFoundError ('{} does not exist'.format(image_path))
    image = torch.from_numpy(process_image(Image.open(image_path))).unsqueeze(0)
    model.double()
    model.to('cpu')
    model.eval()
    output = model(image)
    prob, label_idx = torch.topk(output, topk)
    
    # convert indices to category labels
    k, v = zip(*model.class_to_idx.items())
    idx_to_class = dict(zip(v,k))
    label_classes = [idx_to_class.get(i) for i in label_idx[0].numpy()]
    
    # The model doesn't give probabilities, so use this to do it.
    sm = nn.Softmax(dim=1)
    prob = sm(prob).detach()[0].numpy()
    return prob, label_classes, image.squeeze(0).numpy()

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# Display an image along with the top 5 classes
def show_result(filename, model, class_names=None):
    '''Displays a picture that's predicted with label and top 5 labels
    
    params:  
        filename: str: the path to an image file
        model: pytorch model: trained pytorch model
        cat_map: dict:  lookup for a label number to a label string
    '''
    probs, classes, img_tensor = predict(filename, model)
    if class_names is None:
        names = classes
    else:
        names = [class_names.get(c, c) for c in classes]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4,8))
    
    imshow(img_tensor, ax=ax1, title = names[0])
    
    y_pos = range(len(probs), 0, -1)
    ax2.barh(y_pos, probs)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(names)

In [ ]:
show_result('./flowers/test/78/image_01888.jpg', model, cat_to_name)